In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import warnings
import xgboost
warnings.filterwarnings('ignore')

In [2]:
df_model = pd.read_csv('Data\Crop_Yield_Cleaned.csv')

In [3]:
df_model.head()

,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Days_to_Harvest,Yield_tons_per_hectare,Region_East,Region_North,Region_South,Region_West,...,Soil_Type_Silt,Crop_Barley,Crop_Cotton,Crop_Maize,Crop_Rice,Crop_Soybean,Crop_Wheat,Weather_Condition_Cloudy,Weather_Condition_Rainy,Weather_Condition_Sunny
0,897.077239,27.676966,False,True,122,6.555816,False,False,False,True,...,False,False,True,False,False,False,False,True,False,False
1,992.673282,18.026142,True,True,140,8.527341,False,False,True,False,...,False,False,False,False,True,False,False,False,True,False
2,147.998025,29.794042,False,False,106,1.127443,False,True,False,False,...,False,True,False,False,False,False,False,False,False,True
3,986.866331,16.644190,False,True,146,6.517573,False,True,False,False,...,False,False,False,False,False,True,False,False,True,False
4,730.379174,31.620687,True,True,110,7.248251,False,False,True,False,...,True,False,False,False,False,False,True,True,False,False


In [3]:
df_model.describe()

,Rainfall_mm,Temperature_Celsius,Days_to_Harvest,Yield_tons_per_hectare
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,549.981901,27.504965,104.495025,4.649563
std,259.851320,7.220608,25.953412,1.696325
min,100.000896,15.000034,60.000000,0.000333
25%,324.891090,21.254502,82.000000,3.417637
50%,550.124061,27.507365,104.000000,4.651808
75%,774.738520,33.753267,127.000000,5.879200
max,999.998098,39.999997,149.000000,9.963372


In [4]:
x = df_model.drop(['Yield_tons_per_hectare'],axis=1)
y = df_model['Yield_tons_per_hectare']

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30,random_state=44)
from sklearn.preprocessing import StandardScaler,Normalizer
stdscaler = StandardScaler()
nrml = Normalizer()
X_train = stdscaler.fit_transform(x_train)
X_test = stdscaler.transform(x_test)

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [7]:
##Create a Function to Evaluate Model
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

Initial Model Training

In [8]:
## Beginning Model Training
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "ElasticNet":ElasticNet(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "Adaboost Regressor":AdaBoostRegressor(),
    "Graident BoostRegressor":GradientBoostingRegressor(),
    "Xgboost Regressor":XGBRegressor()
   
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_train_adjusted_r_squared = round(1 - (1-model_train_r2)*(len(y)-1)/(len(y)-x.shape[1]-1),6)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    model_test_adjusted_r_squared = round(1 - (1-model_test_r2)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    print("Adjusted R2 Score : {:.4f}".format(model_train_adjusted_r_squared))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    print("Adjusted R2 Score : {:.4f}".format(model_test_adjusted_r_squared))
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.4997
- Mean Absolute Error: 0.3987
- R2 Score: 0.9132
Adjusted R2 Score : 0.9132
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.5015
- Mean Absolute Error: 0.4005
- R2 Score: 0.9125
Adjusted R2 Score : 0.9125


Lasso
Model performance for Training set
- Root Mean Squared Error: 1.4813
- Mean Absolute Error: 1.2058
- R2 Score: 0.2376
Adjusted R2 Score : 0.2376
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 1.4811
- Mean Absolute Error: 1.2055
- R2 Score: 0.2373
Adjusted R2 Score : 0.2373


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.4997
- Mean Absolute Error: 0.3987
- R2 Score: 0.9132
Adjusted R2 Score : 0.9132
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.5015
- Mean Absolute Error: 0.4005
- R2 Score: 0.9125
Adjusted R2 Score : 0.9125


E

KeyboardInterrupt: 

Hyperparameter Tuning


In [8]:
xgboost_params = {"learning_rate": [0.1, 0.01,1.0,10,100],
                  "max_depth": [5, 8, 12, 20, 30,40,50],
                  "n_estimators": [100, 200, 300,400,500],
                  "colsample_bytree": [0.5, 0.8, 1, 0.3, 0.4]}




In [9]:
# Models list for Hyperparameter tuning
cv_models = [
    
                   ("Xgboost Regressor",XGBRegressor(),xgboost_params),
                   
]


Random CV Hyperparameter tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in cv_models:
    random = RandomizedSearchCV(estimator=model,
                                   param_distributions=params,
                                   n_iter=100,
                                   cv=3,
                                   verbose=2,
                                   n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

In [9]:
## Retraining the models with best parameters with RandomCV
models = {
    "Xgboost Regressor":XGBRegressor(n_estimators= 300, max_depth= 8, learning_rate= 0.01, colsample_bytree= 0.8),
    "GradientBoost":GradientBoostingRegressor(n_estimators= 100, min_samples_split= 15, max_depth=5, loss='huber', learning_rate= 0.1, criterion='squared_error')    
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_train_adjusted_r_squared = round(1 - (1-model_train_r2)*(len(y)-1)/(len(y)-x.shape[1]-1),6)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    model_test_adjusted_r_squared = round(1 - (1-model_test_r2)*(len(y)-1)/(len(y)-x.shape[1]-1),6)
    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    print("Adjusted R2 Score : {:.4f}".format(model_train_adjusted_r_squared))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    print("Adjusted R2 Score : {:.4f}".format(model_test_adjusted_r_squared))
    print('='*35)
    print('\n')

Xgboost Regressor
Model performance for Training set
- Root Mean Squared Error: 0.5196
- Mean Absolute Error: 0.4147
- R2 Score: 0.9062
Adjusted R2 Score : 0.9062
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.5236
- Mean Absolute Error: 0.4181
- R2 Score: 0.9047
Adjusted R2 Score : 0.9047




KeyboardInterrupt: 

Grid CV


In [12]:
from sklearn.model_selection import GridSearchCV

model_param = {}
for name, model, params in cv_models:
    random = GridSearchCV(estimator=model,
                                   param_grid=params,
                                   cv=3,
                                   verbose=2,
                                   n_jobs=-1)
    random.fit(X_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

In [10]:
## Retraining the models with best parameters of Grid CV
models = {
    "Xgboost Regressor":XGBRegressor(n_estimators= 400, max_depth= 5, learning_rate= 0.01, colsample_bytree= 1),
    # "GradientBoost":GradientBoostingRegressor(n_estimators= 500, min_samples_split= 20, max_depth=5, loss='squared_error', learning_rate= 0.01, criterion='friedman_mse'),
   
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    
    print('='*35)
    print('\n')

Xgboost Regressor
Model performance for Training set
- Root Mean Squared Error: 0.5014
- Mean Absolute Error: 0.4001
- R2 Score: 0.9126
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.5041
- Mean Absolute Error: 0.4024
- R2 Score: 0.9117




#Observations
Linear Regression, Ridge, Random Forest, Adaboost, Gradient Boost, XGboost Models Performed Exceptionally well in Initial Evaluation - with best Adjusted R2 Score - 91% with Linear Regression, Ridge and Gradient Boost

To further Enhanced the performance we Hyperparameter tuned the Model with Gradient Boost and Xgboost - Using Random and Grid CV 

Using XGBoostRegressor for final Predictions


======================================================================================
Pickling the XGB Regressor and StandardScalar

In [11]:
#Pickling the XGBoost Model

XGB_Model = XGBRegressor(n_estimators= 400, max_depth= 5, learning_rate= 0.01, colsample_bytree= 1)
XGB_Model.fit(X_train, y_train) # Train model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             device=None, early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.01, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [12]:
import pickle

# Assuming your trained model is stored in the variable 'model'


pickle.dump(XGB_Model,open('XBGR_Model_Agriculture_Yield_Prediction.pkl', 'wb'))

pickle.dump(stdscaler,open('Scalar_Model_Agriculture_Yield_Prediction.pkl', 'wb'))


